# Instalacja pakietów

*   requests - Pakiet Python umożliwiający wysyłanie żądań HTTP w prosty sposób. Umożliwia wykonywanie wszystkich typów żądań HTTP, takich jak GET, POST, PUT, DELETE i inne.
*   chromadb - Baza danych wektorowa służąca do przechowywania i wyszukiwania wektorów. Przydatna do różnych zastosowań związanych z uczeniem maszynowym i przetwarzaniem języka naturalnego.
*   sentence-transformers - Pakiet Python do generowania osadzeń zdań (embeddings) za pomocą różnych modeli transformerowych. Jest używany w zadaniach takich jak wyszukiwanie semantyczne, klasteryzacja, klasyfikacja i inne.
*   PyMuPDF (fitz) - Biblioteka Python umożliwiająca pracę z plikami PDF i innymi formatami dokumentów. Umożliwia odczyt, zapis i manipulację plikami PDF.









In [1]:
!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install requests
!pip install chromadb
!pip install sentence-transformers
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.6 MB/s eta 0:00:

In [2]:
import gc
import torch

import uuid
import json

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Deklaracja ścieżek i nazw plików

In [4]:
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"
data_dir = "./data"

# Pobieranie pliku PDF

In [5]:
import requests

r = requests.get(pdf_url, stream = True)
with open(file_name, "wb") as file:
    for block in r.iter_content(chunk_size = 1024):
         if block:
             file.write(block)

# Ekstrakcja tekstu z pliku PDF

In [6]:
import fitz
import json

document = fitz.open(file_name)
text = ''
pages = []

for page_num in range(len(document)):
    page = document.load_page(page_num)
    text += page.get_text()
    pages.append({"page_num": page_num, "text": page.get_text()})

with open(file_json, "w") as file:
    json.dump(pages, file, indent=4, ensure_ascii=False)

# Stworzenie osadzeń tekstów

In [7]:
from sentence_transformers import SentenceTransformer

texts = []
for page in pages:
    texts.append(page["text"])

embedding_model = SentenceTransformer('ipipan/silver-retriever-base-v1.1')
embeddings = embedding_model.encode(texts)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.35k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Tworzenie bazy wektorów

In [8]:
import chromadb

client = chromadb.PersistentClient(path=data_dir)
collection = client.get_or_create_collection(
    name="ha_naive_rag", metadata={"hnsw:space": "cosine"}
)

collection.add(
    documents=texts,
    embeddings=embeddings,
    ids = [str(i) for i in range(len(texts))]
)


#Zapytajmy chatbota

Jest to prosty bot w stylu **question answering** (**Q&A**) wykorzystujący technikę **RAG** w wersji **tzw. naiwnej**. Pamiętajmy o lepszym chunkowaniu, dokładniejszej analizie struktury dokumentów, wzbogaceniu tekstu, augmentacji zapytań, hybrydowym wyszukiwaniu, dodaniu kontekstu i historii rozmowy, a o**trzymamy PRAWDZIWEGO CHATBOTA w wersji enterprise.** Jednak powyższy kod to kilkanaście linijek kodu, a chatbot przewyższa kilkakrotnie klasyczne chatboty oparte na rozpoznawaniu intencji i zaawansowanych scenariuszach.

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'


temperature = 1.0
max_tokens = 500
top_k = 200
top_p = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=quantization_config
                                            )


tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [19]:
def create_context(query, top = 3):

  embedding = embedding_model.encode([query])

  result = collection.query(
      query_embeddings=embedding,
      n_results= top
  )

  if result:

      ids = result.get("ids", [])
      documents = result.get("documents", [])

      context = "KONTEKST:\n"
      context = context + documents[0][0] + "\n\n"
      if len(documents) > 1:
        context = context + documents[0][1] + "\n\n"

      # zapis do pliku
      filename = f"{str(uuid.uuid4())[:8]}_context.json"

      with open(filename, "w") as file:
          content = {
              "context" : context,
              "query" : query
          }
          json.dump(content, file, ensure_ascii=False, indent=4)

      return f"Wyłącznie na podstawie podanego kontekstu odpowiedz zwięźle na pytanie: '{query}'" +"\n" + context

  else:
    print("Brak kontekstu")
    return None



In [11]:
def generate(prompt, system = None):

  messages = []

  if system:
    messages.append( {"role": "system", "content": system })

  messages.append( {"role": "user", "content": prompt })


  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # generowanie odpowiedzi
  outputs = model.generate(model_inputs,
                               streamer = streamer,
                               max_new_tokens=max_tokens,
                               do_sample=True if temperature else False,
                               temperature = temperature,
                               top_k = top_k,
                               top_p = top_p)

  # zapis do pliku
  filename = f"{str(uuid.uuid4())[:8]}.json"

  with open(filename, "w") as file:
      content = {
          "prompt": messages,
           "output": tokenizer.batch_decode(outputs, skip_special_tokens=False)
      }
      json.dump(content, file, ensure_ascii=False, indent=4)




In [20]:
temperature = 0.1
max_tokens = 1000
top_k = 200
top_p = 1

prompt = create_context(query = "Czy AI zniszczy ludzkość?", top = 3)
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Odpowiedź na pytanie "Czy AI zniszczy ludzkość?" w kontekście podanym zależy od scenariusza, który się zrealizuje. Są trzy scenariusze podane w kontekście:

1. Negatywny scenariusz, w którym AI może wymknąć się spod kontroli ludzkości, co rodzi obawy, że maszyny mogą podejmować decyzje niezgodne z ludzkimi interesami. Taki scenariusz przypomina wizję ludzkości jako zbędnego ogniwa i rodzi obawy Stephena Hawkinga.

2. Neutralny scenariusz, w którym rozwój pełnej sztucznej inteligencji może być zarówno obiecujący, jak i niepokojący. Podkreśla równowagę między wykorzystaniem AI do postępu społecznego a ochroną przed jej potencjalnymi negatywnymi skutkami.

3. Pozytywny scenariusz, w którym AI i ludzkość współpracują, uzupełniając się wzajemnie. Fei-Fei Li, znana badaczka AI, wskazuje, że sztuczna inteligencja może rozszerzać nasze możliwości, a nie zastępować nas. Taka kooperacja otwiera drzwi do nowych odkryć i rozwiązań.

Podsumowując, odpowiedź na pytanie "Czy AI zniszczy ludzkość?" z